# Welcome to the Lab 🥼🧪
## Where do 1000+ unit portfolios own the greatest share of all homes in the country?

In this notebook, we are going to create a great map showing Institutional ownership for single family homes for the entire country. We are going to pull the data for all 3000+ counties.

Welcome to the Lab - LFG

**Note** This notebook will work with any of the 70k+ markets supported by the Parcl Labs API.

As a reminder, you can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along. 

To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY` as a secret. See this [guide](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75) for more information.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-examples/blob/main/python/inspiration/map_of_investor_ownership.ipynb)


You can expect to create this: 

![Chart](../assets/large_institutional_ownership.png)

In [ ]:
# Environment setup
import os
import sys
import json
import subprocess
from datetime import datetime
from urllib.request import urlopen

# Collab setup from one click above
if "google.colab" in sys.modules:
    from google.colab import userdata
    %pip install parcllabs plotly kaleido
    api_key = userdata.get('PARCL_LABS_API_KEY')
else:
    api_key = os.getenv('PARCL_LABS_API_KEY')

In [ ]:
import parcllabs
import pandas as pd
import plotly.express as px
from parcllabs import ParclLabsClient

print(f"Parcl Labs Version: {parcllabs.__version__}")

In [ ]:
client = ParclLabsClient(api_key=api_key)

In [ ]:
# lets get all metros in the country
counties = client.search_markets.retrieve(
    location_type='COUNTY',
    as_dataframe=True,
    params={
        'limit': 1000
    }
)

In [ ]:
# get the ids
county_ids = counties['parcl_id'].tolist()
len(county_ids)

In [ ]:
# How big are the investors? Let's find out
portfolio_size_ownership = client.portfolio_metrics_sf_housing_stock_ownership.retrieve_many(
    parcl_ids=county_ids,
    as_dataframe=True,
    params={
        'limit': 1
    }
)

In [ ]:
# get most recent housing stock for all metros
# gross_yields = client.rental_market_metrics_gross_yield.retrieve_many(
#     parcl_ids=county_ids,
#     property_type='SINGLE_FAMILY',
#     params={
#         'limit': 6 # let's get the most recent stock
#     },
#     as_dataframe=True # make life easy on ourselves
# )

In [ ]:
# merge with search results to get geoid

df = pd.merge(portfolio_size_ownership, counties, on='parcl_id')
df.head()

In [ ]:
df.sort_values('pct_sf_housing_stock_portfolio_1000_plus', ascending=False)[['name', 'state_abbreviation', 'count_portfolio_1000_plus', 'pct_sf_housing_stock_portfolio_1000_plus']].head(20)
                                                                             

In [ ]:
df = df[['count_portfolio_1000_plus', 'pct_sf_housing_stock_portfolio_1000_plus', 'geoid']]

In [ ]:
# lets also get number of single family homes owned by institutions

In [ ]:
# lets create the map
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [ ]:
# rename geoid to fips
df = df.rename(columns={'geoid': 'fips'})
df.head()

In [ ]:
df.head()

In [ ]:
all_fips_ids = [i['id'] for i in counties['features']]
len(all_fips_ids)

In [ ]:
# fill holes with no data
map_ids = df['fips'].unique().tolist()
missing = [i for i in all_fips_ids if i not in map_ids]
missing_df = pd.DataFrame({'fips': missing})
missing_df['count_portfolio_1000_plus'] = 0
missing_df['pct_sf_housing_stock_portfolio_1000_plus'] = 0


out = pd.concat([df, missing_df])
out

In [ ]:
labs_logo_lookup = {
    'blue': 'https://parcllabs-assets.s3.amazonaws.com/powered-by-parcllabs-api.png',
    'white': 'https://parcllabs-assets.s3.amazonaws.com/powered-by-parcllabs-api-logo-white.png'
}

# set charting constants
labs_logo_dict = dict(
        source=labs_logo_lookup['white'],
        xref="paper",
        yref="paper",
        x=0.5,  # Centering the logo below the title
        y=1.02,  # Adjust this value to position the logo just below the title
        sizex=0.15, 
        sizey=0.15,
        xanchor="center",
        yanchor="bottom"
)

media_img_size_lookup = {
    'X': {
        'width': 1600,
        'height': 900
    }
}

PLATFORM = 'X'

# set image sizes
IMG_WIDTH = media_img_size_lookup[PLATFORM]['width']
IMG_HEIGHT = media_img_size_lookup[PLATFORM]['height']

# plotting title settings
PLOT_TITLE_SETTINGS = {
        'y':0,
        'x':0,
        'xanchor': 'left',
        'yanchor': 'bottom'
    }

In [ ]:
# df['pct_sf_housing_stock_portfolio_1000_plus'] = df['pct_sf_housing_stock_portfolio_1000_plus']+100

In [ ]:
# df.head()

In [ ]:
import plotly.express as px
import pandas as pd

# Assuming 'df' and 'counties' are already defined with the correct data.
# Example DataFrame 'df' and counties GeoJSON should be properly defined here.
color_continuous_scale=[
    (0, "#EEF7FF"),
    (0.2, "#B0CFFF"),  
    (1, "#04428C") 
]

fig = px.choropleth(df, geojson=counties, locations='fips', color='pct_sf_housing_stock_portfolio_1000_plus',
                    color_continuous_scale=color_continuous_scale,
                    range_color=(0, 5),
                    scope="usa",
                    labels={'pct_sf_housing_stock_portfolio_1000_plus':'# Homes Owned'},
                    )

# Update layout for background color, margins, and figure size
fig.update_layout(
    title='% of Single Family Homes Owned by Large Institutional Portfolios (1000+ Units)',  # Set the title for the map
    title_x=0.5,  # Center title
    title_font=dict(color='white', size=16),  # Make title font white and set size
    paper_bgcolor='#080D16',  # Dark background color
    geo_bgcolor='#080D16',  # Matching geo background color
    margin={"r":0, "t":30, "l":0, "b":0},  # Adjust top margin to give space for title
    width=800,  # Width of the figure
    height=450,  # Height of the figure
)

# Adjust color bar tick text and title color
fig.update_coloraxes(colorbar=dict(
    title="% of SFH's Owned",
    tickfont=dict(color='white'),
    title_font=dict(color='white', size=12)
))

# Customize boundary borders and centering
fig.update_geos(
    fitbounds='locations',
    visible=False,
    showcountries=True,  # Ensure countries are shown
    countrycolor="white",  # Country border color
    showsubunits=True,  # Ensure subunit boundaries are shown (like states)
    subunitcolor="white",  # Subunits (state) border color
    center=dict(lon=-98.5795, lat=39.8283)  # Center on the middle of the USA
)

# Adding layout image
fig.add_layout_image(
    dict(
        source=labs_logo_lookup['white'],
        xref="paper",
        yref="paper",
        x=0,  # Centering the logo below the title
        y=0,  # Adjust this value to position the logo just below the title
        sizex=0.15, 
        sizey=0.15,
        xanchor="left",
        yanchor="bottom"
)
)

fig.show()
